In [1]:
import io
import pandas as pd
import ibm_boto3
from ibm_botocore.client import Config, ClientError

In [ ]:
# Constants for IBM COS values
COS_ENDPOINT = "<endpoint>"
COS_API_KEY_ID = "<api-key>"
COS_INSTANCE_CRN = "<service-instance-id>"
# Create resource
cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
                        
)

In [ ]:
def get_bucket_contents(bucket_name):
    items_list = []
    print("Retrieving bucket contents from:{0}".format(bucket_name))
    try:
        files = cos.Bucket(bucket_name).objects.all()
        for file in files:
            items_list.append("{0}".format(file.key, file.size))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve bucket contents: {0}".format(e))
    return items_list

In [ ]:
items_list = get_bucket_contents(bucket_name)
df_dict = {}
for item in items_list:
    length = len(item)
    name = item[:length-4] 
    #using the substring before the .csv extension as the name
    key = name
    object = cos.Object('bucket_name', item)
    csvFile = object.get()
    stream = io.StringIO(csvFile["Body"].read().decode('utf-8'))
    df = pd.read_csv(stream)
    df_dict[key] = df